In [3]:
import pandas as pd
import os
import numpy as np
pd.set_option('max_colwidth',100)
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',100)

In [4]:
#设置数据路径，REGCAPCUR.xls意义不明且编码不明，暂时删去
name_list=os.listdir(r'D://cd80//Sample_Data//sample_data_feature')
#读取数据，表名作为变量名
for i in name_list:
    if i[-3:]!='csv':
        continue
    path='D://cd80//Sample_Data//sample_data_feature//'+i
    
    name=i[:-4]
    print(name+',')
    cmd = "%s=pd.read_csv(path)" %name    #定义该变量要赋的值（此处为i）
    exec(cmd)
    eval("%s" %name)


company_ar,
company_ar_alterstockinfo,
company_ar_assetsinfo,
company_ar_nz,
company_ar_socialfee,
company_base_info,
company_modify,
el_company_history_inv,
el_company_history_manager,
tax_abnormal,
tax_company,
tax_qianshui,
tax_year,


In [70]:
#查看每张表中按公司计数的记录条数
print(len(company_ar.entid.value_counts()))
print(len(company_ar_alterstockinfo.entid.value_counts()))
print(len(company_ar_assetsinfo.entid.value_counts()))
print(len(company_ar_nz.entid.value_counts()))
print(len(company_ar_socialfee.entid.value_counts()))
print(len(company_base_info.entid.value_counts()))
print(len(company_modify.entid.value_counts()))
print(len(el_company_history_inv.entid.value_counts()))
print(len(el_company_history_manager.entid.value_counts()))
print(len(tax_abnormal.entid.value_counts()))
print(len(tax_company.entid.value_counts()))
print(len(tax_qianshui.entid.value_counts()))
print(len(tax_year.entid.value_counts()))

2300
253
435
93
2117
2490
1616
639
994
46
1681
28
1660


In [71]:
#以entid作为index，便于后续合并
company_base_info.index=company_base_info.entid
tax_year.index=tax_year.entid
company_ar_assetsinfo.index=company_ar_assetsinfo.entid

In [100]:
#根据entid合并数据集
#由于数据太稀疏()，时间上没有对齐，取第一次出现的记录作为合并数据
temp_df=pd.concat([company_base_info
           ,tax_year.drop_duplicates('entid')
           ,company_ar_assetsinfo.drop_duplicates('entid')],axis=1)[['INDUSTRYPHY','ENTTYPE',
                                                                    'NETINC','ASSGRO','LIAGRO','TOTEQU','incometax_taxable',
                                                                     'finance_expense','revenue','sales_cost','total_profit',
                                                                    'investment_income','fair_value_change','operation_profit',
                                                                    'non_operating_income','non_operating_expense',
                                                                    'sales_tax','sales_expense','G_expense']]
temp_df.isna().sum()

INDUSTRYPHY                 0
ENTTYPE                     0
NETINC                   2103
ASSGRO                   2099
LIAGRO                   2116
TOTEQU                   2118
incometax_taxable        2055
finance_expense          1263
revenue                  1153
sales_cost               1339
total_profit             1072
investment_income        1798
fair_value_change        1817
operation_profit         1071
non_operating_income     1622
non_operating_expense    1633
sales_tax                1384
sales_expense            1514
G_expense                1118
dtype: int64

In [105]:
for i in temp_df.columns:
    if i in ['INDUSTRYPHY','ENTTYPE']:
        continue
    temp_df[i].fillna(temp_df[i].mean(),inplace=True)

In [106]:
radar_df=temp_df

#偿债能力指标
temp_df['A1_ICM_A']=(temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.finance_expense
temp_df['A2_ICM_B']=(temp_df.NETINC+temp_df.finance_expense)/temp_df.finance_expense
temp_df['A3_ALR']=temp_df.LIAGRO/temp_df.ASSGRO
temp_df['A4_Ratio_of_equity_to_debt']=temp_df.TOTEQU/temp_df.LIAGRO
temp_df['A5_EBITDA']=(temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.LIAGRO

temp_df['A']=np.random.rand(2490)

#比率结构指标
temp_df['B1_equity_ratio']=temp_df.TOTEQU/temp_df.ASSGRO
temp_df['B2_Profit_ratio_of_main_business']=(temp_df.revenue-temp_df.sales_cost)/temp_df.total_profit
temp_df['B3_Profit_from_financial_activities']=(temp_df.investment_income+temp_df.fair_value_change)/temp_df.total_profit
temp_df['B4_Operating_profit_ratio']=(temp_df.non_operating_income-temp_df.non_operating_expense)/temp_df.total_profit
temp_df['B5_Flow_rate']=temp_df.sales_tax/temp_df.revenue

temp_df['B']=np.random.rand(2490)

#盈利水平
temp_df['C1_ROA']=temp_df.NETINC/temp_df.ASSGRO

temp_df['C2_ROE']=temp_df.NETINC/temp_df.TOTEQU

temp_df['C3_gross_margin']=(temp_df.revenue-temp_df.sales_cost)/temp_df.revenue

temp_df['C4_Expense_ratio_during_sales']=(temp_df.sales_expense+temp_df.finance_expense+temp_df.G_expense)/temp_df.revenue

temp_df['C5_Operating_margin_before_interest_and_tax']=(temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.revenue

temp_df['C']=np.random.rand(2490)
#风险水平
temp_df['D1_Financial_leverage']=(temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/(temp_df.NETINC+temp_df.incometax_taxable)
temp_df['D2_EM']=temp_df.ASSGRO/temp_df.TOTEQU
temp_df['D3_ER']=temp_df.LIAGRO/temp_df.TOTEQU

temp_df['D']=np.random.rand(2490)
#发展能力

temp_df['E1_Capital_preservation_and_appreciation_rate']=np.random.rand(2490)
temp_df['E2_Growth_rate_of_total_assets']=np.random.rand(2490)
temp_df['E3_Growth_rate_of_revenue']=np.random.rand(2490)

temp_df['E']=np.random.rand(2490)

radar_df=temp_df[['A1_ICM_A','A2_ICM_B','A3_ALR','A4_RED','A5_EBITDA','A',
                  'B1_equity_ratio','B2_Profit_ratio_of_main_business','B3_Profit_from_financial_activities','B4_Operating_profit_ratio','B5_Flow_rate','B',
                  'C1_ROA','C2_ROE','C3_gross_margin','C4_Expense_ratio_during_sales','C5_Operating_margin_before_interest_and_tax','C',
                  'D1_Financial_leverage','D2_EM','D3_ER','D',
                  'E1_Capital_preservation_and_appreciation_rate','E2_Growth_rate_of_total_assets','E3_Growth_rate_of_revenue','E',
                  'INDUSTRYPHY','ENTTYPE'
                 ]]

In [108]:
radar_df.to_csv('radar_df_test.csv')

In [9]:
temp_df.A1_ICM_A.sort_values(ascending=False)

entid
1102521656    3.632476e+08
393529853     1.744734e+08
591231218     1.363237e+08
1156237838    8.903219e+07
1121113591    8.221203e+07
                  ...     
1106278934   -1.105625e+07
908820235    -1.127711e+07
809210755    -1.514301e+07
40212297     -5.144408e+07
855724058    -9.630758e+07
Name: A1_ICM_A, Length: 2490, dtype: float64

In [240]:
tax_year[tax_year.year==2012].isna().sum()

entid                            0
year                             0
islatest                         0
profit_loss                      0
revenue                        143
sales_cost                     281
sales_tax                      293
sales_expense                  377
G_expense                      196
finance_expense                244
asset_devaluation_losses       601
fair_value_change              612
investment_income              594
operation_profit               160
non_operating_income           509
non_operating_expense          457
total_profit                   160
assets                         135
assets_re                      135
payrol_expense                 612
welfare_expenses               612
education_expenses             612
union_funds                    612
ad_expenses                    612
entertainment_expenses         612
assets_loss                    612
asset_devaluation_allowance    612
main_business_income           612
main_business_cost  

In [242]:
temp_df.to_csv('cd80_test.csv')

In [335]:
(tax_company.region_id)

entid
959830672       3
959830672       3
959830672       3
620868429       0
620868429       0
             ... 
330068559    1825
591748203     849
337611555     248
678325399     698
488677666     758
Name: region_id, Length: 5058, dtype: int64

In [97]:
company_base_info.columns

Index(['entid', 'honor_num', 'illegad_num', 'yea_sub', 'web_dum', 'ENTTYPE',
       'INDUSTRYPHY', 'REGCAP', 'REGCAPCUR', 'RECCAP', 'PARNUM', 'LIMPARNUM',
       'PARFORM', 'EXENUM', 'brand_num', 'cancel_dum', 'certificate_num',
       'tax_dum'],
      dtype='object')

In [10]:
company_base_info.REGCAPCUR

0       156.0
1       156.0
2       156.0
3       156.0
4       156.0
        ...  
2485    156.0
2486      NaN
2487    156.0
2488    156.0
2489      NaN
Name: REGCAPCUR, Length: 2490, dtype: float64